# NCSU GIS 582: Geospatial Modeling and Analysis

## Tutorial: 2 - Geospatial Data Models

### Resources

- [GRASS Manual](https://grass.osgeo.org/grass-devel/manuals/)
- [GRASS Jupyter Introduction](https://grass.osgeo.org/grass-devel/manuals/jupyter_intro.html)
- [GRASS Python API Introduction](https://grass.osgeo.org/grass-devel/manuals/python_intro.html)

## Google Colab Setup

Let’s first print system description to know where are we.

In [ ]:
!lsb_release -a

At the time of writing this tutorial, Colab has Linux Ubuntu 22.04.4 LTS. So we add the ppa:ubuntugis repository, update and install GRASS. It might take a couple of minutes according to the resources available.

In [ ]:
!add-apt-repository -y ppa:ubuntugis/ubuntugis-unstable
!apt update
!apt-get install -y grass-core grass-dev

Check that GRASS is installed by asking which version is there.

In [ ]:
!grass --version

Check which Python version is running.

In [ ]:
import sys

v = sys.version_info
print(f"We are using Python {v.major}.{v.minor}.{v.micro}")

## 1. Import Python Packages

Import the Python standard libraries we need.

In [ ]:
import subprocess
import os
from pathlib import Path

We are going to import the GRASS Python API (`grass.script`) and the GRASS Jupyter package (`grass.jupyter`), but first, we need to find the path to those packages using the `--config python_path` command. This command is slightly different for each operating system.

We use `subprocess.check_output` to find the path and `sys.path.append` to add it to the path.

In [ ]:
# Ask GRASS GIS where its Python packages are.
sys.path.append(
    subprocess.check_output(["grass", "--config", "python_path"], text=True).strip()
)

In [ ]:
# Import the GRASS GIS packages we need.
import grass.script as gs
import grass.jupyter as gj

Download and unzip the North Carolina basic sample dataset:

> A complete list of available datasets can be found at: https://grass.osgeo.org/download/data/

In [ ]:
!grass --tmp-project XY --exec g.download.project url=https://grass.osgeo.org/sampledata/north_carolina/nc_basic_spm_grass7.tar.gz path=$HOME

In [ ]:
# Start GRASS Session
session = gj.init("~/grassdata", "nc_spm_08_grass7", "user1")

## Tutorial Tasks

### 1. Resampling raster data to different resolutions

Resample the given raster map to higher and lower resolution (30m->10m, 30m->100m) and compare resampling by nearest neighbor with bilinear and bicubic method.

First, set the computation region extent to our study area and set resolution to 30 meters. The computational region (region for short) is set using g.region module, the -p flag is used to print the region coordinates and resolution. Here we use a predefined region which is included in our data set. 

In [ ]:
!g.region region=swwake_30m -p

Then we display the 30m resolution NED elevation raster. 

In [ ]:
# Create Map instance
elev_ned_30m_map = gj.Map(filename="outputs/elev_ned_30m.png")
elev_ned_30m_map.d_rast(map="elev_ned_30m@PERMANENT")
elev_ned_30m_map.d_legend(raster="elev_ned_30m@PERMANENT", at=[90,50,5,8])

# Display map
elev_ned_30m_map.show()

To resample it to 10m resolution, first set the computational region to resolution 10m, then resample the raster using the nearest neighbor method.

> Hint: To open the r.resamp.interp in GUI, type or paste the module name into the Console tab, then Enter to open the GUI dialog, don't forget to set the method to nearest under Optional tab. 

In [ ]:
%%bash
g.region res=10 -p
r.resamp.interp elev_ned_30m out=elev_ned10m_nn method=nearest

The resulting elevation map should display automatically, if not, display the resampled map by adding "elev_ned10m_nn" to Layer Manager in case you don't have it in the Layer Manager already.

Alternatively, use in command line the following: 

In [ ]:
# Create Map instance
elev_ned10m_nn_map = gj.Map(filename="outputs/elev_ned10m_nn.png")
elev_ned10m_nn_map.d_rast(map="elev_ned10m_nn")
elev_ned10m_nn_map.d_legend(raster="elev_ned10m_nn", at=[90,50,5,8])

# Display map
elev_ned10m_nn_map.show()

The elevation map "elev_ned10m_nn" looks the same as the original one, so now check the resampled elevation surface using the aspect map: 

In [ ]:
!r.slope.aspect elevation=elev_ned10m_nn aspect=aspect_ned10m_nn

 You should see the aspect map derived from the resampled elevation map in your display, or you can add "aspect_ned10m_nn" to Layer Manager or in command line using: 

#### Map: Aspect Nearest Neighbor Interoplation

In [ ]:
# Create Map instance
aspect_ned10m_nn_map = gj.Map(filename="outputs/aspect_nn.png")
aspect_ned10m_nn_map.d_rast(map="aspect_ned10m_nn")
aspect_ned10m_nn_map.d_legend(raster="aspect_ned10m_nn", at=[90,50,5,8])

# Display map
aspect_ned10m_nn_map.show()

Now, reinterpolate DEMs using bilinear and bicubic interpolation. Check the structure of interpolated elevation surfaces using aspect maps. 

In [ ]:
%%bash
r.resamp.interp elev_ned_30m out=elev_ned10m_bil meth=bilinear
r.resamp.interp elev_ned_30m out=elev_ned10m_bic meth=bicubic
r.slope.aspect elevation=elev_ned10m_bil aspect=aspect_ned10m_bil
r.slope.aspect elevation=elev_ned10m_bic aspect=aspect_ned10m_bic

#### Map: Aspect Bilinear Interoplation

In [ ]:
# Create Map instance

aspect_ned10m_bil_map = gj.Map(filename="outputs/aspect_bil.png")
aspect_ned10m_bil_map.d_rast(map="aspect_ned10m_bil")
aspect_ned10m_bil_map.d_legend(raster="aspect_ned10m_bil", at=[90,50,5,8])

# Display map
aspect_ned10m_bil_map.show()

#### Map: Aspect Bicubic Interoplation

In [ ]:
# Create Map instance

aspect_ned10m_bic_map = gj.Map(filename="outputs/aspect_bic.png")
aspect_ned10m_bic_map.d_rast(map="aspect_ned10m_bic")
aspect_ned10m_bic_map.d_legend(raster="aspect_ned10m_bic", at=[90,50,5,8])

# Display map
aspect_ned10m_bic_map.show()

#### Question 1

**Why is the aspect map of elevation raster map computed by the nearest neighbor method different from the one computed by bilinear interpolation?**

Resample to lower resolution (30m -> 100m).

Then change the region resolution and resample elevation (which is a continuous field) and land use (which has discrete categories).

#### Question 2

**Explain selection of aggregation method. Can we use average also for landuse? What does mode mean?**

In [ ]:
%%bash
g.region res=100 -p
r.resamp.stats elev_ned_30m out=elev_new100m_avg method=average

#### Map: Elevation 100m

In [ ]:
# Create Map instance
elev_100m_map = gj.Map(filename="outputs/elev_100m.png")
elev_100m_map.d_rast(map="elev_new100m_avg")
elev_100m_map.d_legend(raster="elev_new100m_avg", at=[90,50,5,8])

# Display map
elev_100m_map.show()

Resample landuse to 100m resolution

In [ ]:
!r.resamp.stats landuse96_28m out=landuse96_100m method=mode

#### Map: Landuse 100m

In [ ]:
# Create Map instance
landuse96_100m_map = gj.Map(filename="outputs/lenduse_100m.png")
landuse96_100m_map.d_rast(map="landuse96_100m")

# Display map
landuse96_100m_map.show()

### 2. Converting between vector data types

Convert census blocks polygons to points using their centroids (useful for interpolating a population density trend surface): 

In [ ]:
!v.to.points census_wake2000 type=centroid out=census_centr use=vertex

#### Map: Vector to Points

In [ ]:
# Create Map instance
vect_to_points_map = gj.Map(filename="outputs/vect_to_points.png")
vect_to_points_map.d_vect(map="census_wake2000", color="red", fill_color=None)
vect_to_points_map.d_vect(map="census_centr", icon="basic/circle", fill_color="green", size=10)

vect_to_points_map.d_legend_vect(at=[80, 15])

# Display map
vect_to_points_map.show()

### Convert from vector to raster

Convert vector data to raster for use in raster-based analysis. First, remove all previous map layers from Layer Manager and adjust the computational region to resolution 200m: 

In [ ]:
!g.region swwake_30m res=200 -p

Convert vector points "schools" to raster. As value for raster use attribute column "CORECAPACI" for core capacity.

In [ ]:
%%bash
v.info -c schools_wake
v.to.rast schools_wake out=schools_cap_200m use=attr attrcol=CORECAPACI type=point

#### Map: Schools to raster

In [ ]:
# Create Map instance
schools_wake_map = gj.Map(filename="outputs/schools_cap_200m.png")

# Display raster data
schools_wake_map.d_rast(map="schools_cap_200m")

# Display vector data
schools_wake_map.d_vect(map="schools_wake", color="red", fill_color=None)
schools_wake_map.d_vect(map="streets_wake", color="grey")

# Display raster legend
schools_wake_map.d_legend(raster="schools_cap_200m", at=[70,30,2,6])

# Display map
schools_wake_map.show()

Now convert lines in "streets" vector to raster. Set the resolution to 30m and use speed limit attribute. 

In [ ]:
%%bash
g.region res=30 -p
v.to.rast streets_wake out=streets_speed_30m use=attr attrcol=SPEED type=line

#### Map: Vector to Raster

In [ ]:
# Create Map instance
vect_to_rast_map = gj.Map(filename="outputs/vect_to_rast.png")

# Display raster data
vect_to_rast_map.d_rast(map="streets_speed_30m")

# Display raster legend
vect_to_rast_map.d_legend(raster="streets_speed_30m", at=[5,30,2,5], use=[25,35,45,55,65] )

# Display map
vect_to_rast_map.show()

### Convert from raster to vector data

Convert raster lines to vector lines.

First, remove map layers from Layer Manager and set the region. Hint: right click on the raster legends to select Remove legend.

Then do the conversion. 

In [ ]:
%%bash 
g.region raster=streams_derived -p
d.rast streams_derived
r.thin streams_derived output=streams_derived_t
r.to.vect streams_derived_t output=streams_derived_t type=line

#### Question 3

**Explain why we are using r.thin module.**

 Visually compare the result with streams digitized from airphotos. To make space for the vector legend you may need to stretch your Map Display window a little wider. 

#### Map: Streams Compare

In [ ]:
# Create Map instance
streams_compare_map = gj.Map(filename="outputs/streams_compare.png")

# Display vector data
streams_compare_map.d_vect(map="streams_derived_t", color="blue", fill_color=None)
streams_compare_map.d_vect(map="streams", color="red")

# Display raster legend
streams_compare_map.d_legend_vect()

# Display map
streams_compare_map.show()

Convert raster areas representing basins to vector polygons.

Verify the basin boundaries by displaying them together with streams - the stream networks should "fit" within the basin boundaries. 

In [ ]:
%%bash
g.region raster=basin_50K -p
r.to.vect -sv basin_50K output=basin_50Kval type=area

#### Map: Basins

In [ ]:
# Create Map instance
basins_map = gj.Map(filename="outputs/basins.png")

# Display raster data
basins_map.d_rast(map="basin_50K")

# Display vector data
basins_map.d_vect(map="basin_50Kval", type="boundary", width=2)
basins_map.d_vect(map="streams", color="blue")

# Display raster legend
basins_map.d_legend(raster="schools_cap_200m", at=[70,30,2,6])

# Display map
basins_map.show()

## 2B - Data display and visualization

Resources:

* GRASS GIS overview and manual
* Recommendations how to use GUI from the first assignment


### Changing the default font

Change the default font used for map rendering in GUI Settings: File > Preferences > Map Display. Pick a new default font and Save the settings.
Note, that you can also change the default color table, default vector display properties and other display behavior in GUI settings. 

### Visualization in 3D perspective

Interactively view "elevation" raster and vector data ("streams", "usgsgages") in 3D view, adjust viewing position, surface properties and lighting to highlight features and display landuse data over topography. Note, that legend is not currently supported in 3D view. Remove or switch off any map layers in the Layer Manager (if you have any) and set region to the raster "elevation". 

In [ ]:
!g.region raster=elevation -p 

Then add raster "elevation", and vector "streams", and "usgsgages".
Zoom to computational region and Switch to 3D view.

Follow the video Visualization in GRASS I: surface and Visualization in GRASS II: draping points and lines and save 2-3 images for your report (save as tiff or take screenshot, if you don't have tiff support). 

#### Map: 3D Elevation

In [ ]:
elevation_3dmap = gj.Map3D(filename="outputs/3dElevation.png")
# Full list of options m.nviz.image
# https://grass.osgeo.org/grass83/manuals/m.nviz.image.html
elevation_3dmap.render(
    elevation_map="elevation",
    color_map="elevation",
    perspective=20,
    height=3000,
    vline="streams",
    vline_color="blue",
    vpoint="usgsgages",
    vpoint_color="red",
    vpoint_marker="sphere",
    vpoint_size=200,
    fringe=['ne','nw','sw','se'],
    arrow_position=[100,50],
)
elevation_3dmap.overlay.d_legend(raster="elevation", at=(60, 97, 87, 92))
elevation_3dmap.show()

### Visualizing multiple surfaces

Visualize multiple surfaces (bare earth and surface with vegetation and structures), analyze their relationship using crossections generated by interactive cutting planes.

It is recommended to quit GRASS before starting the task below because we will be working with a smaller, high resolution region.

Start grass with your previous mapset. First set region to "rural_1m", then interpolate surface with vegetation from multiple return lidar points using the module v.surf.rst (we will explain interpolation later on). 

In [ ]:
%%bash
g.region rural_1m -p
v.surf.rst input=elev_lidrural_mrpts elevation=elev_lidruralmr_1m npmin=120 segmax=25 tension=40 smooth=1.0

In [ ]:
elevation_3dmap = gj.Map3D(filename="outputs/3dElevationMultiSurface.png")
# Full list of options m.nviz.image
# https://grass.osgeo.org/grass83/manuals/m.nviz.image.html
elevation_3dmap.render(
    elevation_map="elev_lidruralmr_1m,elev_lid792_1m",
    color_map="elev_lidruralmr_1m,elev_lid792_1m",
    # color="green,brown",
    mode="fine,fine",
    resolution_fine="1,1",
    resolution_coarse="9,9",
    style="surface,surface",
    shading="gouraud,gouraud",
    wire_color="black,black",
    surface_position=["0,0,190", "0,0,0"],
   
    perspective=20,
    height=1900,
    position=[1, 1]
)

elevation_3dmap.overlay.d_legend(raster="elevation", at=(60, 97, 87, 92))
elevation_3dmap.show()

### Basic 2D display operations

GUI is recommended for the tasks below, see the GUI equivalents for selected d* commands, the command line instructions below are to indicate the workflow and output.

**Display subsets of data**

Visualy explore relation between developed areas and topography. Remove all previously used layers from the Layer Manager, then set region and display land use categories 1, 2 (developed land) over shaded topography.
Settings > Region > Set region > Set region to match this raster > select "landuse96_28m".

Add raster > select "elevation_shade".
Add raster > select "landuse96_28m" > Selection > List of cats to display > 1,2.

Zoom to computational region.

Right click on "landuse96_28m" map layer and Change opacity to show topography blended with landuse. Save display to graphic file. 

#### Map: landuse elevation

In [ ]:
!g.region raster=landuse96_28m -p

In [ ]:
# Create Map instance
landuse_elev_map = gj.Map(filename="outputs/landuse_elev.png")

# Display raster data
landuse_elev_map.d_rast(map="elevation_shade")
landuse_elev_map.d_rast(map="landuse96_28m", values=[1,2])

# Display map
landuse_elev_map.show()

### Change colors for raster maps

There are many ways how to adjust or create custom color ramps for raster maps, see r.colors manual, we explore only some basic tools here.

First create your own copy of the elevation map (see the command below).
Display it by Add raster > "myelev". 

In [ ]:
!g.copy raster=elevation,myelev

Set the color table to a predefined one (feel free to select a different one than the suggestion below, e.g. sepia).

GUI options: Right Click on the raster layer "myelev" in Layer Manager > Set color table > Define > Name of color table > "elevation" > Run.
If you don't see the new colors - click the first button in Map Display to redraw. 

#### Map: MyNewElevation

In [ ]:
# Change the color scheme to elevation
!r.colors myelev color=elevation

# Create Map instance
mynewelevation_map = gj.Map(filename="outputs/mynewelevation.png")

# Display raster data
mynewelevation_map.d_rast(map="myelev")

# Display map
mynewelevation_map.show()

To create color table for future multiple uses, especially for scripting, type the rules into a plain text mycolor.txt file using a text editor (TextEdit, Notepad) and save it in your working directory, for example (feel free to use your own colors):

Change the colors using your custom rules

In [ ]:
!r.colors myelev rules=inputs/mycolor.txt

#### Map: mynewelevation2

In [ ]:
# Create Map instance
mynewelevation2_map = gj.Map(filename="outputs/mynewelevation2.png")

# Display raster data
mynewelevation2_map.d_rast(map="myelev")

# Display map
mynewelevation2_map.show()

Note that you can combine color RGB and name definitions as well as % and cat/val to create complex custom color tables and store them in a text file for future use, see the examples in r.colors man page. 

### Compare the use of equal interval and histogram equalized color table for slope

 First, we set an equal interval color ramp to our copy of the slope map. To add the legend use Add map elements button on Map Display. 

In [ ]:
%%bash
g.copy raster=slope,myslope
r.colors myslope color=bgyr

#### Map: Slope Color

# Create Map instance
myslope_map = gj.Map(filename="outputs/myslopecolor.png")

# Display raster data
myslope_map.d_rast(map="myslope")

myslope_map.d_legend(raster="myslope", at=[70,30,2,6])

# Display map
myslope_map.show()

Now set the histogram equalized color table, and save the new slope map. 

In [ ]:
!r.colors -e myslope color=bgyr

#### Map: Slope Color Histogram Equalized

In [ ]:
# Create Map instance
myslope_map = gj.Map(filename="outputs/myslopecolorequalized.png")

# Display raster data
myslope_map.d_rast(map="myslope")

myslope_map.d_legend(raster="myslope", at=[70,30,2,6])

# Display map
myslope_map.show()

To explain the difference between the two maps, you can generate a histogram. On Map Display click on Analyze map > Create histogram to open the histogram tool and save results to graphics file. Or use this command which will add a histogram as a layer to Layer Manager and then save the image: 

#### Histogram: Slope Color Histogram Equalized

In [ ]:
slopehistogram = gj.Map(filename="outputs/slopehistogram.png")
slopehistogram.d_histogram(map="myslope")
slopehistogram.show()

#### Question 4

**What is the effect of the histogram equalized color table on the slope map pattern?**

### Modify legend, scale and grid

To re-size the legend for myslope, right click on the legend, select Resize legend and resize with mouse. Alternatively you can resize by selecting Pointer mode in Map Display, double clicking on legend to launch the legend dialog and set options > Optional > Placement > 50,90,4,7.
Numbers are bottom,top,left,right as percentage of screen coordinates.
Add units to the legend: Add map elements > Add text layer > type deg > OK.
Add barscale on Map Display: Add map elements > Add scale bar (double click on it to change iti, for example you can change its length and units under Optional tab).

Note: you can use horizontal legends by using Placement at=6,10,2,30 or just stretch it horizontally with mouse. 

In [ ]:
# Create Map instance
myslope_map = gj.Map()

# Display raster data
myslope_map.d_rast(map="myslope")
myslope_map.d_barscale(length=1000)
myslope_map.d_legend(raster="myslope", at=[6,10,20,50])

# Display map
myslope_map.show()

#### Map: Elevation Map with Grid

In [ ]:
# Create Map instance
myelevmap_map = gj.Map(filename="outputs/myelevmap.png")

# Display raster data
myelevmap_map.d_rast(map="myelev")
myelevmap_map.d_grid(size=5000, color="brown")
myelevmap_map.d_grid(size=1000, flags="n")
myelevmap_map.d_grid(size="0:02", color="black", flags="g")
myelevmap_map.d_legend(raster="myelev")

# Display map
myelevmap_map.show()

### Display color composite

#### Map: RGB Landsat

In [ ]:
# Create Map instance
mylandsat_map = gj.Map(filename="outputs/mylandsat.png")

mylandsat_map.d_rgb(
    red="lsat7_2002_30",
    green="lsat7_2002_20",
    blue="lsat7_2002_10"
)

mylandsat_map.d_vect(map="roadsmajor", color="yellow")

# Display map
mylandsat_map.show()

### Export NC precipitation stations data to KML and post in Google Earth

To see what you are exporting, add the precip_30ynormals raster or use the following command: 

In [ ]:
# Create Map instance
mylandsat_map = gj.Map()

mylandsat_map.d_rgb(
    red="lsat7_2002_30",
    green="lsat7_2002_20",
    blue="lsat7_2002_10"
)

mylandsat_map.d_vect(map="roadsmajor", color="yellow")
mylandsat_map.d_vect(map="precip_30ynormals", color="red")
# Display map
mylandsat_map.show()


You may need to use right click in Layers to zoom to the extent of the whole vector map. Then use v.out.ogr to export the vector map in the KML format. If your are running the command in GUI do not forget to click point for data type. To view the data in Google Earth just drag and drop the KML file in the Google Earth window.
Make sure you include extension ".kml" in the output file name.

In [ ]:
%%bash
v.out.ogr precip_30ynormals output=precipitation.kml format=KML type=point